TensorRT is a machine learning framework that is published by Nvidia to run inference that is machine learning inference on their hardware. TensorRT is highly optimized to run on NVIDIA GPUs. It's likely the fastest way to run a model at the moment.

> **TensorRT** — фреймворк, который разрабатывается компанией Nvidia, а значит, позволяет разработчикам учесть нюансы аппаратного устройства вычислителей.
> 

В нём модель оптимизируется перед запуском таким образом, чтобы для вычисления каждой операции использовались самые быстрые алгоритмы на конкретном железе с определённым количеством свободной памяти (именно в целевом приложении).



Перед отправкой модели в TensorRT нужно конвертировать её в ONNX — это удобный способ передачи модели между фреймворками. TensorRT поддерживает и динамическое построение модели через создание **NetworkDefinition**. Некоторые фреймворки (например, [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt)) используют такую функцию для вычисления в TensorRT частей модели (другие вычисляются на PyTorch). Но обычно это медленнее и связано с наличием у модели операций, которые не поддерживаются TensorRT или ONNX.

Полезная вещь для ONNX-модели — применение оптимизаций из **onnx-simplifier** или **onnx-graphsurgeon**. PyTorch конвертирует модель в ONNX операция за операцией, а эти библиотеки распознают их паттерны, заменяют на более простые аналоги, не нарушая семантику вычислительного графа. Пример оптимизации с помощью onnx-simplifier представлен на картинке ниже. Перед запуском TensorRT для упрощения ему работы рекомендуется применить onnx-simplifier к вашему целевому onnx.

In [20]:
import torch
import torchvision
import torch_tensorrt

# Option 1: torch.compile

In [21]:
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)

In [22]:
# define model
model.eval().cuda()

# define input
x = torch.randn((1, 3, 224, 224)).cuda()

In [23]:
%timeit
optimized_model = torch.compile(model=model, backend='tensorrt')
optimized_model(x)

INFO:torch_tensorrt.dynamo.utils:Using Default Torch-TRT Runtime (as requested by user)
INFO:torch_tensorrt.dynamo.utils:Device not specified, using Torch default current device - cuda:0. If this is incorrect, please specify an input device, via the device keyword.
INFO:torch_tensorrt.dynamo.utils:Compilation Settings: CompilationSettings(enabled_precisions={<dtype.f32: 7>}, debug=False, workspace_size=0, min_block_size=5, torch_executed_ops=set(), pass_through_build_failures=False, max_aux_streams=None, version_compatible=False, optimization_level=None, use_python_runtime=False, truncate_double=False, use_fast_partitioner=True, enable_experimental_decompositions=False, device=Device(type=DeviceType.GPU, gpu_id=0), require_full_compilation=False, disable_tf32=False, assume_dynamic_shape_support=False, sparse_weights=False, refit=False, engine_capability=<EngineCapability.STANDARD: 1>, num_avg_timing_iters=1, dla_sram_size=1048576, dla_local_dram_size=1073741824, dla_global_dram_size=53

tensor([[-5.0904e-01,  2.1708e-01,  4.9388e-01,  1.1321e-01,  1.0922e-01,
         -8.5650e-02, -6.7061e-01, -3.3510e-01, -3.3576e-02,  5.3123e-01,
          2.1692e+00,  2.3855e+00,  3.1600e+00,  1.2145e+00,  2.3697e+00,
          2.5355e+00,  3.1621e+00,  7.3838e-01,  3.5306e+00,  1.5613e+00,
          9.3568e-01,  5.1983e+00,  4.6880e+00,  4.2450e+00,  1.5544e+00,
         -6.2847e-01, -5.2865e-01, -4.0366e-01, -4.0650e-01, -4.0630e-01,
         -6.0638e-01,  3.1652e-01, -6.6534e-01, -7.3297e-01, -4.6542e-01,
         -8.4165e-01, -2.9450e-01, -9.5617e-01, -2.2987e-01, -3.5871e-01,
          3.1342e-01, -6.5971e-01,  2.4042e-01,  4.4494e-02, -4.0216e-01,
         -4.8257e-01,  2.1305e-01,  1.0862e-01, -6.0703e-01, -5.7100e-01,
         -2.5311e-02, -6.9373e-01, -3.2193e-01, -5.9247e-01, -2.5352e-01,
          2.5007e-01, -2.9238e-01, -5.1614e-01, -4.1690e-01,  1.7109e+00,
          8.7341e-02, -8.7084e-01, -4.8313e-01,  1.6086e-01,  5.5343e-01,
         -5.9889e-01, -6.4109e-01, -3.

In [24]:
%%timeit
optimized_model(x)

2.95 ms ± 41.8 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
%%timeit
model(x)

13.2 ms ± 907 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Option2. Export 

C++

Step 1: Optimize + serialize

In [32]:
inputs = [torch.randn((1, 3, 224, 224)).cuda()]

In [33]:
trt_gm = torch_tensorrt.compile(model, inputs=inputs)
torch_tensorrt.save(trt_gm, "trt.ts", output_format="torchscript", inputs=inputs) # for C++ inference(C++ support only torchscript)

INFO:torch_tensorrt._compile:ir was set to default, using dynamo frontend
INFO:torch_tensorrt.dynamo._compiler:Compilation Settings: CompilationSettings(enabled_precisions={<dtype.f32: 7>}, debug=False, workspace_size=0, min_block_size=5, torch_executed_ops=set(), pass_through_build_failures=False, max_aux_streams=None, version_compatible=False, optimization_level=None, use_python_runtime=False, truncate_double=False, use_fast_partitioner=True, enable_experimental_decompositions=False, device=Device(type=DeviceType.GPU, gpu_id=0), require_full_compilation=False, disable_tf32=False, assume_dynamic_shape_support=False, sparse_weights=False, refit=False, engine_capability=<EngineCapability.STANDARD: 1>, num_avg_timing_iters=1, dla_sram_size=1048576, dla_local_dram_size=1073741824, dla_global_dram_size=536870912, dryrun=False, hardware_compatible=False)

INFO:torch_tensorrt [TensorRT Conversion Context]:[MemUsageChange] Init CUDA: CPU +0, GPU +0, now: CPU 6651, GPU 1521 (MiB)
INFO:torch_te

Step 2: Deploy

Deployment in C++:

In [29]:

#include "torch/script.h"
#include "torch_tensorrt/torch_tensorrt.h"

# auto trt_mod = torch::jit::load("trt.ts");
# auto input_tensor = [...]; // fill this with your inputs
# auto results = trt_mod.forward({input_tensor});

Python

In [37]:
model = model.eval().cuda()
inputs = [torch.randn((1, 3, 224, 224)).cuda()]
# trt_ep is a torch.fx.GraphModule object
trt_gm = torch_tensorrt.compile(model, ir="dynamo", inputs=inputs)
torch_tensorrt.save(trt_gm, "trt.ep", inputs=inputs)

In [ ]:
# Inference
model = torch.export.load("trt.ep").module()
model(*inputs)